<a href="https://colab.research.google.com/github/meti-94/TextGeneration/blob/main/bert_autoencoder_personachat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
!tar -xf  simple-examples.tgz
!mkdir data
!mv /content/simple-examples/data/ptb.train.txt data/
!mv /content/simple-examples/data/ptb.valid.txt data/
!mv /content/simple-examples/data/ptb.test.txt data/
!rm -rf ./simple_examples

In [1]:
!pip install transformers
!wget https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
!mkdir models

     |████████████████████████████████| 2.1MB 23.5MB/s 
     |████████████████████████████████| 901kB 51.8MB/s 
     |████████████████████████████████| 3.3MB 50.2MB/s 
--2021-04-20 14:04:40--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.48.254
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.48.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘personachat_self_original.json’

personachat_self_or 100%[===================>] 200.13M  54.5MB/s    in 3.7s    

2021-04-20 14:04:44 (53.9 MB/s) - ‘personachat_self_original.json’ saved [209850483/209850483]



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
from torch.utils.data import Dataset, DataLoader
from transformers import EncoderDecoderModel, BertTokenizer, BertModel, BertConfig
import torch
from transformers import AdamW
from tqdm import tqdm
import json
import random 
from sklearn.model_selection import train_test_split
from datetime import datetime
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from tqdm import tqdm_notebook as tqdm
import logging
logging.basicConfig(level=logging.DEBUG)

In [43]:
SPECIAL_TOKENS = ["<bos>", "<eos>", "<persona>", "<speaker1>", "<speaker2>", "<pad>"]

ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>', '<persona>']}

encoder_max_length=512
decoder_max_length=128

def read_data(data_json_path='/content/personachat_self_original.json'):
  with open(data_json_path) as json_file:
    data_dict = json.load(json_file)
  return data_dict

def data_to_samples(data_dict, test=False):
  samples=[]
  for dialogue in (data_dict['train'] if test==False else data_dict['valid']):
    original_persona = dialogue['personality']
    
    for item in dialogue['utterances']:
      original_persona = [original_persona[-1]] + original_persona[:-1]
      history = item['history']
      response = item['candidates'][-1]
      samples.append({
          'persona':original_persona,
          'history':history,
          'response':response
      })
  return samples
  
def bertified(samples):
  bertified_data = []
  for item in samples:
    persona = ' <persona> '.join(item['persona'])
    persona = '<bos> ' + persona
    history = ''
    speakers = [" <speaker1> ", " <speaker2> "]
    speaker = 0
    for hst in item['history'][::-1]:
      history = speakers[speaker] + hst + history
      speaker = 1 - speaker
    response = '<speaker2> ' + item['response'] + ' <eos>'
    bertified_data.append({
          'persona':persona.replace('  ', ' '),
          'history':history.replace('  ', ' '),
          'input': persona.replace('  ', ' ')+' '+history.replace('  ', ' '), 
          'response':response.replace('  ', ' ')
      })
  return bertified_data

def bert_sample_tokenizer(bert_sample, tokenizer):
  inputs = tokenizer(bert_sample['input'], add_special_tokens=True, max_length=encoder_max_length, truncation=True, padding="max_length")
  outputs = tokenizer(bert_sample['response'], add_special_tokens=True, max_length=decoder_max_length, truncation=True, padding="max_length")
  _item = {}
  _item["input_ids"] = inputs.input_ids
  _item["attention_mask"] = inputs.attention_mask
  _item["decoder_input_ids"] = outputs.input_ids
  _item["decoder_attention_mask"] = outputs.attention_mask
  _item["labels"] = outputs.input_ids.copy()
  return _item 

In [3]:
|# print(type(test), type(test[0]), test[0])
# print(type(test_data_samples), test_data_samples[0])

In [4]:
# encoder_max_length=512
# decoder_max_length=128

# def process_data_to_model_inputs(batch):
#   # tokenize the inputs and labels
#   inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
#   outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

#   batch["input_ids"] = inputs.input_ids
#   batch["attention_mask"] = inputs.attention_mask
#   batch["decoder_input_ids"] = outputs.input_ids
#   batch["decoder_attention_mask"] = outputs.attention_mask
#   batch["labels"] = outputs.input_ids.copy()

#   # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
#   # We have to make sure that the PAD token is ignored
#   batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

#   return batch

In [41]:
class PersonaDataset_v3(Dataset):
  '''
      Convert Data to proper Tensor dataset
  '''
  def __init__(self, samples):
    self.samples = samples
    self.n_samples = len(self.samples)

  def __getitem__(self, index):
    # returns specific item
    return self.samples[index]

  def __len__(self):
    return self.n_samples

In [36]:
# X = self.tokenizer(batch['input'], add_special_tokens=True, max_length=512, truncation=True, padding=True)
# y = self.tokenizer(batch['response'], add_special_tokens=True, max_length=100, truncation=True, padding=True)
# X = torch.tensor(X["input_ids"])
# y = torch.tensor(y['input_ids'])
# encoder_max_length=512
# decoder_max_length=128
# class PersonaDataset_v2(Dataset):
#   '''
#       Convert Data to proper Tensor dataset
#   '''
#   def __init__(self, samples):
#     self.samples = samples
#     self.n_samples = len(self.samples)

#   def __getitem__(self, index):
#     # returns specific item
#     inputs = tokenizer(self.samples[index]['input'], add_special_tokens=True, max_length=encoder_max_length, truncation=True, padding="max_length")
#     outputs = tokenizer(self.samples[index]['response'], add_special_tokens=True, max_length=decoder_max_length, truncation=True, padding="max_length")
#     _item = {}
#     _item["input_ids"] = inputs.input_ids
#     _item["attention_mask"] = inputs.attention_mask
#     _item["decoder_input_ids"] = outputs.input_ids
#     _item["decoder_attention_mask"] = outputs.attention_mask
#     _item["labels"] = outputs.input_ids.copy()
#     return _item
#   def __len__(self):
#     return self.n_samples
    # returns dataset length

In [6]:
# class PersonaDataset_v1(Dataset):
#   '''
#       Convert Data to proper Tensor dataset
#   '''
#   def __init__(self, samples):
#     self.samples = samples
#     self.n_samples = len(self.samples)

#   def __getitem__(self, index):
#     # returns specific item
#     return self.samples[index] 
#   def __len__(self):
#     return self.n_samples
#     # returns dataset length


# class PTBDataset(Dataset):
#   '''
#       Convert Data to proper Tensor dataset
#   '''
#   def __init__(self, path):
#     self.texts = []
#     with open(path, 'r') as fin:
#       for line in fin:
#         self.texts.append(line.strip())
#     self.n_samples = len(self.texts)

#   def __getitem__(self, index):
#     # returns specific item
#     return self.texts[index] 
#   def __len__(self):
#     return self.n_samples
#     # returns dataset length





In [55]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("prajjwal1/bert-tiny", "prajjwal1/bert-tiny")
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN)
model.get_encoder().resize_token_embeddings(len(tokenizer))
model.get_decoder().resize_token_embeddings(len(tokenizer))

data = read_data()
data_samples = data_to_samples(data)
bertified_data = bertified(data_samples)
train, valid = train_test_split(bertified_data, test_size=0.15, random_state=99)
test_data = read_data()
test_data_samples = data_to_samples(test_data, True)
test = bertified(test_data_samples)

one_line_tokenizer = lambda bert_sample:bert_sample_tokenizer(bert_sample, tokenizer)

# train_dataset = PersonaDataset_v3(list(zip(*map(one_line_tokenizer, tqdm(train)))))
valid_dataset = PersonaDataset_v3(list(map(one_line_tokenizer, tqdm(train))))
valid_dataset = PersonaDataset_v3(list(map(one_line_tokenizer, tqdm(valid))))
test_dataset = PersonaDataset_v3(list(map(one_line_tokenizer, tqdm(test))))



DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /prajjwal1/bert-tiny/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationsh

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [54]:
valid_dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'])

In [56]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [57]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

# model.config.max_length = 142
# model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [58]:
batch_size = 128
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./",
    logging_steps=2,
    save_steps=10,
    eval_steps=4,
    # logging_steps=1000,
    # save_steps=500,
    # eval_steps=7500,
    # warmup_steps=2000,
    # save_total_limit=3,
)
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)
trainer.train()

TypeError: ignored

In [ ]:
class TrainingLoop:
  '''
  Everything related to model training
  '''
  def __init__( self, model, tokenizer, optimizer, freezeemb=True, 
                epochs=6, save_path='./models/', **kw):
    self.model = model
    params = []
    for paramname, param in self.model.named_parameters():
      if paramname.startswith("bert.embeddings.word_embeddings"):
        if not freezeemb:
          params.append(param)
      else:
        params.append(param)
    self.optimizer = optimizer(params, **kw)
    self.tokenizer = tokenizer
    self.epochs = epochs
    self.save_path = save_path
    self.predicts = None


  def train(self, dataloader, eval_dataloader, test_dataloader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.model = self.model.to(device)
    for epoch in range(self.epochs):
      self.model.train()
      losses = []

      for _, batch in enumerate(tqdm(dataloader, position=0, leave=True, desc=f"Train Epoch Number {epoch+1}")):
        self.model.zero_grad()
        X = self.tokenizer(batch['input'], add_special_tokens=True, max_length=512, truncation=True, padding=True)
        y = self.tokenizer(batch['response'], add_special_tokens=True, max_length=100, truncation=True, padding=True)
        X = torch.tensor(X["input_ids"])
        y = torch.tensor(y['input_ids'])
        X = X.to(device); y = y.to(device)
        outputs = self.model(input_ids=X, decoder_input_ids=y, labels=y) 
        losses.append(outputs.loss.detach().item())
        outputs.loss.backward()
        self.optimizer.step()
        # break 
      logging.info(f'Epoch number: {epoch+1} Train Loss is equal: {sum(losses)/len(losses)}') 
      self.random_predict(test_dataloader, device, number_of_samples=10)
      self.eval(eval_dataloader, epoch, device)
      self.save(f"/content/drive/MyDrive/models/autoencoder_{epoch}_{datetime.today().strftime('%Y-%m-%d')}.pt")


  def eval(self, dataloader, epoch, device):
    self.model = self.model.to(device)
    self.model.eval()
    losses = []
    for _, batch in enumerate(tqdm(dataloader, position=0, leave=True, desc=f"Eval Epoch Number {epoch+1}")):
      with torch.no_grad():
        X = self.tokenizer(batch['input'], add_special_tokens=True, max_length=512, truncation=True, padding='longest')
        y = self.tokenizer(batch['response'], add_special_tokens=True, max_length=100, truncation=True, padding='longest')
        X = torch.tensor(X["input_ids"])
        y = torch.tensor(y['input_ids'])
        X = X.to(device); y = y.to(device)
        outputs = self.model(input_ids=X, decoder_input_ids=y, labels=y) 
        
        losses.append(outputs.loss.detach().item())
        # break
    logging.info(f'Epoch number: {epoch+1} Eval Loss is equal: {sum(losses)/len(losses)}')
  
  def save(self, save_path='./models/autoencoder.pt'):
    logging.info(f'Saving model ...')
    torch.save(self.model, save_path)
	
  def load(self, save_path='./models/autoencoder.pt'):
    logging.info(f'Loading model ...')
    self.model = torch.load(save_path)

  def random_predict(self, dataloader, device, number_of_samples=10):
    counter=0
    for sample in dataloader:
      counter+=1
      _input = self.tokenizer(sample['input'], add_special_tokens=True, max_length=512, padding=True)
      _input = torch.tensor(_input['input_ids'])
      _input = _input.to(device)
      self.model = self.model.to(device)
      decoder_start = torch.tensor(30526).to(device)
      generated = self.model.generate(_input, decoder_start_token_id=torch.tensor(30526).to(device))
      logging.info('Real: '+ sample['response'][0])
      logging.info(tokenizer.convert_ids_to_tokens(generated[0]))
      if counter>number_of_samples:
        break 


In [ ]:
# 

model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN)
model.get_encoder().resize_token_embeddings(len(tokenizer))
model.get_decoder().resize_token_embeddings(len(tokenizer))

optimizer = AdamW
kw = {'lr':0.0005, 'weight_decay':0.1}
tl = TrainingLoop(model, tokenizer, optimizer, False, **kw)
# tl.load('/content/drive/MyDrive/models/autoencoder_0_2021-03-27.pt')
data = read_data()
data_samples = data_to_samples(data)
bertified_data = bertified(data_samples)
train, valid = train_test_split(bertified_data, test_size=0.15, random_state=99)
test_data = read_data()
test_data_samples = data_to_samples(test_data, True)
test = bertified(test_data_samples)

train_dataset = PersonaDataset_v1(train)
valid_dataset = PersonaDataset_v1(valid)
test_dataset = PersonaDataset_v1(test)

train_dataloader = DataLoader(train_dataset, batch_size=6, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=6, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tl.train(train_dataloader, valid_dataloader, test_dataloader)



# tl.save()
# ##################################################
# tl.load()
# tl.random_predict(test_dataloader, device, number_of_samples=10)
# ##################################################
# tl.readable_predict(device, print_result=True)